In [118]:
# Стандартные библиотеки
import pandas as pd
import requests
import numpy as np

# Сторонние библиотеки
import datetime
import re


TODO:
- гиперссылки в содержании некорректно работают
- объкдинить в один # Средневзвешенная процентная ставка по ипотечным кредитам, региональный разрез (%)
- обновить данные до 2025 года

# Содержание
* [1. Данные по ипотечным ставкам](#chapter1)
* [2. Данные по ценам на недвижимость](#chapter2)
* [3. Макроэкономические данные](#chapter3)
* [4. Предобработка данных](#chapter4)

## 1. Данные по ипотечным ставкам <a class="anchor" id="chapter1"></a>

Источник: ЦБ РФ  
Период: 2013–2024 (11 лет)

- Ключевая ставка ЦБ РФ (%) - https://cbr.ru/hd_base/KeyRate/
- Cредний уровень процентной ставки по ипотечному кредиту, региональный разрез (%) - https://www.fedstat.ru/indicator/59319  
- Ставка по кредиту на квартиру с разбиением по типу ипотеки, при условиях (первоначальный взнос=30%, срок = 15 лет)  - https://дом.рф/analytics/mortgage/
- Cредневзвешенная процентная ставка по ипотечным жилищным кредитам в рублях, региональный разрез с разбиением по типу недвижимости, % - https://www.fedstat.ru/indicator/60293, https://cbr.ru/statistics/bank_sector/mortgage/?utm_source=w&utm_content=page
- Количество ипотечных жилищных кредитов, предоставленных физическим лицам-резидентам в рублях, региональный разрез  с разбиением по типу недвижимости, единиц - https://cbr.ru/statistics/bank_sector/mortgage/?utm_source=w&utm_content=page
- Объем жилищных кредитов в рублях, предоставленных физическим лицам-резидентам, региональный разрез  с разбиением по типу недвижимости, млн руб - https://cbr.ru/statistics/bank_sector/mortgage/?utm_source=w&utm_content=page


In [122]:
start_date =  datetime.date(2010, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.datetime.today().strftime('%d.%m.%Y')

# Ключевая ставка 
url_key_rate = f'https://cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'

response = requests.get(url_key_rate)
response.raise_for_status()

key_rate_df = pd.read_html(url_key_rate)[0]

key_rate_df.columns = ['date', 'key_rate']
key_rate_df['key_rate'] = key_rate_df['key_rate']/100

key_rate_df

,date,key_rate
0,10.03.2025,21.0
1,07.03.2025,21.0
2,06.03.2025,21.0
3,05.03.2025,21.0
4,04.03.2025,21.0
...,...,...
2867,23.09.2013,5.5
2868,20.09.2013,5.5
2869,19.09.2013,5.5
2870,18.09.2013,5.5


In [134]:
# Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)

file_path = r'..\data\excel\home_average_mortgage_rate_region.xls'
home_mortgage_avg_rate_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_mortgage_avg_rate_df.columns = ['region','year', 'month', 'avg_rate_cbr']
home_mortgage_avg_rate_df['market_type'] = 'total'

home_mortgage_avg_rate_df


,region,year,month,avg_rate_cbr,market_type
0,Российская Федерация,2018,значение показателя за год,9.56,total
1,Российская Федерация,2019,январь,9.88,total
2,Российская Федерация,2019,февраль,10.04,total
3,Российская Федерация,2019,март,10.18,total
4,Российская Федерация,2019,апрель,10.28,total
...,...,...,...,...,...
7487,Чукотский автономный округ,2024,август,8.38,total
7488,Чукотский автономный округ,2024,сентябрь,8.57,total
7489,Чукотский автономный округ,2024,октябрь,8.86,total
7490,Чукотский автономный округ,2024,ноябрь,8.85,total


Ставка ЦБ РФ не совсем отражает реальность, поэтому используем дополнительный источник — данные от ДОМ.РФ

In [135]:
# Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (доли)
# Динамика ставок предложения топ-20 ипотечных банков

file_path = r'..\data\excel\dom_rf_mortgage_rate.xlsx'
dom_rf_home_mortgage_avg_rate_df = pd.read_excel(file_path, skiprows=1)

dom_rf_home_mortgage_avg_rate_df = dom_rf_home_mortgage_avg_rate_df.drop('Unnamed: 0', axis=1)
dom_rf_home_mortgage_avg_rate_df = dom_rf_home_mortgage_avg_rate_df.drop('Unnamed: 2', axis=1)

collection_df_list = []

for i in range(0, len(dom_rf_home_mortgage_avg_rate_df)):   
    selected_rows = dom_rf_home_mortgage_avg_rate_df.iloc[i, :]
    selected_rows = selected_rows.transpose().reset_index()
    type_of_housing_market = selected_rows.iloc[0, 1]
    selected_rows = selected_rows.drop([0, 216])
    selected_rows['type_of_housing_market'] = type_of_housing_market
    selected_rows.columns = ['date', 'avg_rate_dom_rf', 'type_of_housing_market']
    collection_df_list.append(selected_rows)

dom_rf_home_mortgage_avg_rate_df = pd.concat(collection_df_list, axis=0, ignore_index=True)

dom_rf_home_mortgage_avg_rate_df

,date,avg_rate_dom_rf,type_of_housing_market
0,2020-12-25 00:00:00,0.0791,Новостройка
1,2021-01-08 00:00:00,0.0791,Новостройка
2,2021-01-15 00:00:00,0.079,Новостройка
3,2021-01-22 00:00:00,0.079,Новостройка
4,2021-01-29 00:00:00,0.0786,Новостройка
...,...,...,...
1070,2025-02-07 00:00:00,0.283199,Рефинансирование
1071,2025-02-14 00:00:00,0.2837,Рефинансирование
1072,2025-02-21 00:00:00,0.2837,Рефинансирование
1073,2025-02-28 00:00:00,0.2837,Рефинансирование


In [136]:
# Для обработки следующих excel таблиц
def preprocessing_region_excel_df(df: pd.DataFrame,name_column: str) -> pd.DataFrame:
    """
    Обрабатывает DataFrame, преобразуя его в длинный формат с добавлением столбца 'region'

    Параметры:
    - df: Исходный DataFrame.
    - name_column: Название столбца, которое будет использоваться для значений

    Возвращает:
    - Обработанный DataFrame
    """
    collection_df_list = []

    for i in range(0, len(df)):   
        selected_rows = df.iloc[i, :]
        selected_rows = selected_rows.transpose().reset_index()
        region = selected_rows.iloc[0, 1]
        selected_rows = selected_rows.drop(0)
        selected_rows['region'] = region
        selected_rows.columns = ['month', name_column, 'region']
        collection_df_list.append(selected_rows)

    df = pd.concat(collection_df_list, axis=0, ignore_index=True)
    return df



In [139]:
# Средневзвешенная процентная ставка по ипотечным кредитам, региональный разрез с разбиением по типу недвижимости (%)

file_path = r'..\data\downloaded_xlsx\02_08_Rates_housing.xlsx'
home_weight_mortgage_avg_rate_df = pd.read_excel(file_path, sheet_name='ставка в рублях', skiprows=2)
home_weight_mortgage_avg_rate_df = preprocessing_region_excel_df(home_weight_mortgage_avg_rate_df, name_column='avg_rate')
home_weight_mortgage_avg_rate_df['type_of_housing_market'] = 'total'

home_weight_mortgage_avg_rate_df

C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,avg_rate,region,type_of_housing_market
0,Январь 2019,9.89,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
1,Февраль 2019,10.16,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
2,Март 2019,10.42,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
3,Апрель 2019,10.56,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
4,Май 2019,10.54,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
...,...,...,...,...
7003,Сентябрь 2024,10.37,Чукотский автономный округ,total
7004,Октябрь 2024,12.1,Чукотский автономный округ,total
7005,Ноябрь 2024,8.78,Чукотский автономный округ,total
7006,Декабрь 2024,10.4,Чукотский автономный округ,total


`primary` - первичный жилой рынок, новостройки (под залог прав требования по договорам участия в долевом строительстве)

In [140]:
file_path = r'..\data\downloaded_xlsx\02_17_Rates_scpa_mortgage.xlsx'
home_weight_mortgage_avg_rate_primary_df = pd.read_excel(file_path, sheet_name='ставка в рублях', skiprows=1)
home_weight_mortgage_avg_rate_primary_df = preprocessing_region_excel_df(home_weight_mortgage_avg_rate_primary_df, name_column='avg_rate')
home_weight_mortgage_avg_rate_primary_df['type_of_housing_market'] = 'primary'

home_weight_mortgage_avg_rate_primary_df

C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,avg_rate,region,type_of_housing_market
0,Январь 2019,9.65,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
1,Февраль 2019,9.94,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
2,Март 2019,10.09,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
3,Апрель 2019,10.13,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
4,Май 2019,10.04,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
...,...,...,...,...
7003,Сентябрь 2024,5.97,Чукотский автономный округ,primary
7004,Октябрь 2024,5.98,Чукотский автономный округ,primary
7005,Ноябрь 2024,8.84,Чукотский автономный округ,primary
7006,Декабрь 2024,5.35,Чукотский автономный округ,primary


`secondary` - вторичный жилой рынок, приобретение и создание объектов объектов индивидуального жилищного строительства

In [142]:
file_path = r'..\data\downloaded_xlsx\02_45_Rates_mortgage_ihc.xlsx'
home_weight_mortgage_avg_rate_secondary_df = pd.read_excel(file_path, sheet_name='ставка в рублях', skiprows=2)
home_weight_mortgage_avg_rate_secondary_df = preprocessing_region_excel_df(home_weight_mortgage_avg_rate_secondary_df, name_column='avg_rate')
home_weight_mortgage_avg_rate_secondary_df['type_of_housing_market'] = 'secondary'

home_weight_mortgage_avg_rate_secondary_df

C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,avg_rate,region,type_of_housing_market
0,Январь 2024,7.96,РОССИЙСКАЯ ФЕДЕРАЦИЯ,secondary
1,Февраль 2024,8.24,РОССИЙСКАЯ ФЕДЕРАЦИЯ,secondary
2,Март 2024,8.12,РОССИЙСКАЯ ФЕДЕРАЦИЯ,secondary
3,Апрель 2024,7.99,РОССИЙСКАЯ ФЕДЕРАЦИЯ,secondary
4,Май 2024,8.07,РОССИЙСКАЯ ФЕДЕРАЦИЯ,secondary
...,...,...,...,...
1243,Сентябрь 2024,14.33,Чукотский автономный округ,secondary
1244,Октябрь 2024,15.87,Чукотский автономный округ,secondary
1245,Ноябрь 2024,10.42,Чукотский автономный округ,secondary
1246,Декабрь 2024,14.42,Чукотский автономный округ,secondary


In [143]:
# Количество ипотечных жилищных кредитов, предоставленных физическим лицам-резидентам в рублях, 
# региональный разрез с разбиением по типу недвижимости, единиц

file_path = r'..\data\downloaded_xlsx\02_10_Quantity_mortgage.xlsx'
home_mortgage_cnt_df = pd.read_excel(file_path, sheet_name='в рублях', skiprows=2)
home_mortgage_cnt_df = preprocessing_region_excel_df(home_mortgage_cnt_df, name_column='mortgage_cnt')
home_mortgage_cnt_df['type_of_housing_market'] = 'total'

home_mortgage_cnt_df


C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,mortgage_cnt,region,type_of_housing_market
0,Январь 2019,81054,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
1,Февраль 2019,105570,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
2,Март 2019,103614,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
3,Апрель 2019,110319,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
4,Май 2019,86917,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
...,...,...,...,...
7003,Сентябрь 2024,42,Чукотский автономный округ,total
7004,Октябрь 2024,42,Чукотский автономный округ,total
7005,Ноябрь 2024,24,Чукотский автономный округ,total
7006,Декабрь 2024,21,Чукотский автономный округ,total


In [144]:
file_path = r'..\data\downloaded_xlsx\02_15_Quantity_scpa_mortgage.xlsx'
home_mortgage_cnt_primary_df = pd.read_excel(file_path, sheet_name='в рублях', skiprows=2)
home_mortgage_cnt_primary_df = preprocessing_region_excel_df(home_mortgage_cnt_primary_df, name_column='mortgage_cnt')
home_mortgage_cnt_primary_df['type_of_housing_market'] = 'primary'

home_mortgage_cnt_primary_df

C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,mortgage_cnt,region,type_of_housing_market
0,22845,7114,ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,primary
1,28794,9005,ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,primary
2,27961,8521,ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,primary
3,28830,8747,ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,primary
4,24261,7261,ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,primary
...,...,...,...,...
6930,32053,12,Чукотский автономный округ,primary
6931,32612,8,Чукотский автономный округ,primary
6932,30053,7,Чукотский автономный округ,primary
6933,35599,11,Чукотский автономный округ,primary


In [145]:
file_path = r'..\data\downloaded_xlsx\02_42_Quantity_mortgage_ihc.xlsx'
home_mortgage_cnt_seconadry_df = pd.read_excel(file_path, sheet_name='в рублях', skiprows=2)
home_mortgage_cnt_seconadry_df = preprocessing_region_excel_df(home_mortgage_cnt_seconadry_df, name_column='mortgage_cnt')
home_mortgage_cnt_seconadry_df['type_of_housing_market'] = 'seconadry'

home_mortgage_cnt_seconadry_df

C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,mortgage_cnt,region,type_of_housing_market
0,Январь 2024,22416,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
1,Февраль 2024,28475,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
2,Март 2024,32809,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
3,Апрель 2024,36863,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
4,Май 2024,45801,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
...,...,...,...,...
1243,Сентябрь 2024,10,Чукотский автономный округ,seconadry
1244,Октябрь 2024,5,Чукотский автономный округ,seconadry
1245,Ноябрь 2024,5,Чукотский автономный округ,seconadry
1246,Декабрь 2024,2,Чукотский автономный округ,seconadry


In [146]:
# Объем ипотечных жилищных кредитов в рублях, предоставленных физическим лицам-резидентам, 
# региональный разрез с разбиением по типу недвижимости, млн руб

file_path = r'..\data\downloaded_xlsx\02_11_New_loans_mortgage.xlsx'
home_mortgage_volume_df = pd.read_excel(file_path, sheet_name='в рублях', skiprows=2)
home_mortgage_volume_df = preprocessing_region_excel_df(home_mortgage_volume_df, name_column='mortgage_volume')
home_mortgage_volume_df['type_of_housing_market'] = 'total'
home_mortgage_volume_df

C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,mortgage_volume,region,type_of_housing_market
0,Январь 2019,173119,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
1,Февраль 2019,230600,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
2,Март 2019,227491,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
3,Апрель 2019,242515,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
4,Май 2019,192107,РОССИЙСКАЯ ФЕДЕРАЦИЯ,total
...,...,...,...,...
7003,Сентябрь 2024,175,Чукотский автономный округ,total
7004,Октябрь 2024,161,Чукотский автономный округ,total
7005,Ноябрь 2024,95,Чукотский автономный округ,total
7006,Декабрь 2024,85,Чукотский автономный округ,total


In [147]:
file_path = r'..\data\downloaded_xlsx\02_16_New_loans_scpa_mortgage.xlsx'
home_mortgage_volume_primary_df = pd.read_excel(file_path, sheet_name='в рублях', skiprows=1)
home_mortgage_volume_primary_df = preprocessing_region_excel_df(home_mortgage_volume_primary_df, name_column='mortgage_volume')
home_mortgage_volume_primary_df['type_of_housing_market'] = 'primary'
home_mortgage_volume_primary_df

C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,mortgage_volume,region,type_of_housing_market
0,Январь 2019,57334,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
1,Февраль 2019,74259,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
2,Март 2019,72955,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
3,Апрель 2019,76036,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
4,Май 2019,65352,РОССИЙСКАЯ ФЕДЕРАЦИЯ,primary
...,...,...,...,...
7003,Сентябрь 2024,75,Чукотский автономный округ,primary
7004,Октябрь 2024,36,Чукотский автономный округ,primary
7005,Ноябрь 2024,39,Чукотский автономный округ,primary
7006,Декабрь 2024,61,Чукотский автономный округ,primary


In [148]:
file_path = r'..\data\downloaded_xlsx\02_43_New_loans_mortgage_ihc.xlsx'
home_mortgage_volume_seconadry_df = pd.read_excel(file_path, sheet_name='в рублях', skiprows=2)
home_mortgage_volume_seconadry_df = preprocessing_region_excel_df(home_mortgage_volume_seconadry_df, name_column='mortgage_volume')
home_mortgage_volume_seconadry_df['type_of_housing_market'] = 'seconadry'
home_mortgage_volume_seconadry_df

C:\Users\Mi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,month,mortgage_volume,region,type_of_housing_market
0,Январь 2024,62034,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
1,Февраль 2024,76791,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
2,Март 2024,90849,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
3,Апрель 2024,101531,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
4,Май 2024,111017,РОССИЙСКАЯ ФЕДЕРАЦИЯ,seconadry
...,...,...,...,...
1243,Сентябрь 2024,30,Чукотский автономный округ,seconadry
1244,Октябрь 2024,17,Чукотский автономный округ,seconadry
1245,Ноябрь 2024,16,Чукотский автономный округ,seconadry
1246,Декабрь 2024,1,Чукотский автономный округ,seconadry


## 2. Данные по ценам на недвижимость <a class="anchor" id="chapter2"></a>

Источник: Росстат   
Период: 2014-2024

- Средняя цена за квадратный метр (руб) -  https://www.fedstat.ru/indicator/31452
- Индекс цен на рынке жилья по регионам , региональный разрез (%) - https://www.fedstat.ru/indicator/30925


In [ ]:
# TODO тип рынка Средняя цена 1 кв. м общей площади квартир на рынке жилья  (рубль)

file_path = r'..\data\excel\home_average_price_region.xls'
home_average_price_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_average_price_df.columns = ['type_of_housing_market','region','year', 'quarter', 'price']

home_average_price_df

,type_of_housing_market,region,year,quarter,price
0,Вторичный рынок жилья,Российская Федерация,2014,I квартал,55924.70
1,Вторичный рынок жилья,Российская Федерация,2014,II квартал,56630.42
2,Вторичный рынок жилья,Российская Федерация,2014,III квартал,57119.36
3,Вторичный рынок жилья,Российская Федерация,2014,IV квартал,58084.71
4,Вторичный рынок жилья,Российская Федерация,2015,I квартал,58707.41
...,...,...,...,...,...
8029,Первичный рынок жилья,Чукотский автономный округ,2024,III квартал,233799.46
8030,Первичный рынок жилья,Крымский федеральный округ,2016,I квартал,51250.42
8031,Первичный рынок жилья,Крымский федеральный округ,2016,II квартал,50877.34
8032,Первичный рынок жилья,Крымский федеральный округ,2016,III квартал,51239.18


In [ ]:
# TODO тип рынка Индексы цен на рынке жилья (%)

file_path = r'..\data\excel\home_index_price_region.xls'
home_index_price_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=3)
home_index_price_df.columns = ['type_of_housing_market', 'region', 'year', 'quarter', 'index']

home_index_price_df

,type_of_housing_market,region,year,quarter,index
0,Вторичный рынок жилья,Российская Федерация,2014,I квартал,103.64
1,Вторичный рынок жилья,Российская Федерация,2014,II квартал,103.53
2,Вторичный рынок жилья,Российская Федерация,2014,III квартал,104.12
3,Вторичный рынок жилья,Российская Федерация,2014,IV квартал,105.12
4,Вторичный рынок жилья,Российская Федерация,2015,I квартал,104.80
...,...,...,...,...,...
7839,Первичный рынок жилья,Чукотский автономный округ,2023,II квартал,100.00
7840,Первичный рынок жилья,Чукотский автономный округ,2023,III квартал,100.00
7841,Первичный рынок жилья,Чукотский автономный округ,2023,IV квартал,100.00
7842,Первичный рынок жилья,Чукотский автономный округ,2024,I квартал,112.77


## 3. Макроэкономические данные <a class="anchor" id="chapter3"></a>

Источник: Росстат, ЦБ РФ  
Период: 2014-2024

- Уровень инфляции (%) - https://cbr.ru/statistics/ddkp/infl/?utm_source=w&utm_content=page
- Среднедушевые денежные доходы населения, региональный разрез (руб)
- Курс доллара (руб)


In [155]:
start_date =  datetime.date(2014, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d.%m.%Y')

# Ключевая ставка + Инфляция

url = f'https://cbr.ru/statistics/ddkp/infl/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'

response = requests.get(url)
response.raise_for_status()

key_rate_inflation_df = pd.read_html(url)[0]

key_rate_inflation_df.columns = ['date', 'key_rate', 'inflation', 'target_inflation']
key_rate_inflation_df['key_rate'] = key_rate_inflation_df['key_rate']/100
key_rate_inflation_df['inflation'] = key_rate_inflation_df['inflation']/100
key_rate_inflation_df = key_rate_inflation_df.drop('target_inflation', axis=1)

key_rate_inflation_df

,date,key_rate,inflation
0,12.2024,21.0,9.52
1,11.2024,21.0,8.88
2,10.2024,21.0,8.54
3,9.2024,19.0,8.63
4,8.2024,18.0,9.05
...,...,...,...
127,5.2014,7.5,7.59
128,4.2014,7.5,7.33
129,3.2014,7.0,6.92
130,2.2014,5.5,6.21


In [152]:
# Среднедушевые денежные доходы населения

file_path = r'..\data\excel\avg_monetary_income_people_region.xlsx'

avg_monetary_income_people_df = pd.read_excel(file_path, skiprows=2)
avg_monetary_income_people_df = avg_monetary_income_people_df.drop([97, 98, 99, 100, 101])

# Таблица представлена не очень удобном формате, поэтому обработаем ее
collection_df_list = []

for i in range(1, len(avg_monetary_income_people_df)):   
    selected_rows = avg_monetary_income_people_df.iloc[[0, i], :]
    selected_rows = selected_rows.transpose()
    region = selected_rows.iloc[0, 1]
    selected_rows = selected_rows.drop('Unnamed: 0')
    selected_rows = selected_rows.reset_index()
    selected_rows['region'] = region
    selected_rows.columns = ['year', 'quarter', 'income', 'region']
    collection_df_list.append(selected_rows)

avg_monetary_income_people_df = pd.concat(collection_df_list, axis=0, ignore_index=True)
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].replace(r'Unnamed:*', pd.NA, regex=True)
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].ffill()

avg_monetary_income_people_df.info()
avg_monetary_income_people_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6144 entries, 0 to 6143
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   year     6144 non-null   object
 1   quarter  6048 non-null   object
 2   income   6112 non-null   object
 3   region   6144 non-null   object
dtypes: object(4)
memory usage: 192.1+ KB


,year,quarter,income,region
0,2011\nгод,NaN,20771.642488,Российская Федерация
1,2012 год,I кв.,19105.67146,Российская Федерация
2,2012 год,II кв.,22572.96167,Российская Федерация
3,2012 год,III кв.,23262.024407,Российская Федерация
4,2012 год,IV кв.,27963.833212,Российская Федерация
...,...,...,...,...
6139,2023 год,IV кв. 2),156092,Чукотский авт.округ
6140,2023 год,год 3),156988,Чукотский авт.округ
6141,2024 год 2),I кв.,147290,Чукотский авт.округ
6142,2024 год 2),II кв.,157555,Чукотский авт.округ


In [154]:
# Курс доллара
start_date =  datetime.date(2014, 1, 1).strftime('%d/%m/%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d/%m/%Y')

xml_url = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235'

us_rate_df = pd.read_xml(xml_url)

us_rate_df

,Date,Id,Nominal,Value,VunitRate
0,01.01.2014,R01235,1,"32,6587","32,6587"
1,10.01.2014,R01235,1,"33,1547","33,1547"
2,11.01.2014,R01235,1,"33,2062","33,2062"
3,14.01.2014,R01235,1,"33,1204","33,1204"
4,15.01.2014,R01235,1,"33,2386","33,2386"
...,...,...,...,...,...
2710,25.12.2024,R01235,1,"99,8729","99,8729"
2711,26.12.2024,R01235,1,"99,6125","99,6125"
2712,27.12.2024,R01235,1,"99,2295","99,2295"
2713,28.12.2024,R01235,1,"100,5281","100,5281"


## 4. Предобработка данных <a class="anchor" id="chapter4"></a>

### Этапы предобработки данных:
- **Приведение столбцов с датами к единому формату**  
- **Удаление шума, исправление ошибок и устранение некорректных значений**  
- **Объединение данных из разных источников в единый набор**  
- **Преобразование данных в формат CSV для дальнейшей обработки**  

### Текущие наборы данных:
- **key_rate_df** — Ключевая ставка (%)
- **home_average_mortgage_rate_df** — Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)
- **home_weight_avg_mortgage_rate_df** — Средневзвешенная процентная ставка по ипотечным кредитам, региональный разрез (%)
- **home_average_price_df** — Средняя цена 1 кв. м общей площади квартир на рынке жилья, региональный разрез (рубль)
- **home_index_price_df** — Индексы цен на рынке жилья, региональный разрез (%)
- **key_rate_inflation_df** — Инфляция (%)
- **avg_monetary_income_people_df** — Среднедушевые денежные доходы населения, региональный разрез (рубль)
- **us_rate_df** — Курс доллара, стоимость 1 доллара (рубль)



### 1) Работа со столбцом даты

- **Преобразуем столбец даты в формат `datetime`**
- **Убедимся, что все датасеты имеют единый диапазон дат**
- **Заполним пропущенные значения (NaN)**

In [20]:
# компоненты, необъодимые для предобработки
def datetime_year_month_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует DataFrame:
    - Удаляет строки с 'значение показателя за год'
    - Создаёт столбец 'date' в формате datetime64[ns] с использованием словаря месяцев
    - Удаляет столбцы 'year' и 'month'
    
    Args:
        df (pd.DataFrame): Исходный DataFrame с колонками 'year', 'month'
    
    Returns:
        pd.DataFrame: Обновлённый DataFrame с новым столбцом 'date'
    """

    month_dict = {
        'январь': '01', 'февраль': '02', 'март': '03', 'апрель': '04', 'май': '05',
        'июнь': '06', 'июль': '07', 'август': '08', 'сентябрь': '09', 'октябрь': '10',
        'ноябрь': '11', 'декабрь': '12'
    }
    df = df[df['month'] != 'значение показателя за год'].copy()
    df.loc[:, 'date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].map(month_dict) + '-01')
    df = df.drop(columns=['year', 'month'])

    return df

def datetime_year_quarter_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует DataFrame:
    - Создаёт столбец 'date' в формате datetime64[ns], используя year и quarter
    - Удаляет столбцы 'year' и 'quarter'
    
    Args:
        df (pd.DataFrame): Исходный DataFrame с колонками 'year', 'quarter'
    
    Returns:
        pd.DataFrame: Обновлённый DataFrame с новым столбцом 'date'
    """

    quarter_dict = {
        'I квартал': '01', 'II квартал': '04', 'III квартал': '07', 'IV квартал': '10'
    }
    df = df.copy()
    df.loc[:, 'date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['quarter'].map(quarter_dict) + '-01')
    df = df.drop(columns=['year', 'quarter'])

    return df

# для понимания диапазона дат в каждом датасетие и выявления общего диапазона дат
min_date_list = [] # Список самой ранней даты по каждому набору
max_date_list = [] # Список самой поздней даты по каждому набору

In [ ]:
# key_rate_df

# Создадим копию датафрейма для обработки, чтобы изменения не отражались на исходном наборе и мы могли откатится обратно
key_rate_df_copy = key_rate_df.copy()

key_rate_df_copy['date'] = pd.to_datetime(key_rate_df_copy['date'], format='%d.%m.%Y')

full_date_range = pd.date_range(start=key_rate_df_copy['date'].min(), end=key_rate_df_copy['date'].max(), freq='D')
full_date_df = pd.DataFrame(full_date_range, columns=['date'])

missing_dates = full_date_range.difference(key_rate_df_copy['date'])
print("Пропущенные даты:")
print(missing_dates)

Пропущенные даты:
DatetimeIndex(['2013-09-21', '2013-09-22', '2013-09-28', '2013-09-29',
               '2013-10-05', '2013-10-06', '2013-10-12', '2013-10-13',
               '2013-10-19', '2013-10-20',
               ...
               '2024-11-24', '2024-11-30', '2024-12-01', '2024-12-07',
               '2024-12-08', '2024-12-14', '2024-12-15', '2024-12-21',
               '2024-12-22', '2024-12-29'],
              dtype='datetime64[ns]', length=1297, freq=None)


Пропущенные даты - это выходные дни, заполним эти пропущенные ячейки значениями из соседних ячеек сверху

In [14]:
key_rate_df_copy = key_rate_df_copy.merge(full_date_df, how='outer', on='date')
key_rate_df_copy.isna().sum()

date           0
key_rate    1297
dtype: int64

Заполним значения `key_rate` в новых ячейках соседними значениями

In [21]:
key_rate_df_copy = key_rate_df_copy.ffill()
print(key_rate_df_copy.isna().sum())
min_date_list.append(min(key_rate_df_copy['date']))
max_date_list.append(max(key_rate_df_copy['date']))
key_rate_df_copy

date        0
key_rate    0
dtype: int64


,date,key_rate
0,2013-09-17,5.5
1,2013-09-18,5.5
2,2013-09-19,5.5
3,2013-09-20,5.5
4,2013-09-21,5.5
...,...,...
4118,2024-12-26,21.0
4119,2024-12-27,21.0
4120,2024-12-28,21.0
4121,2024-12-29,21.0


In [22]:
# home_average_mortgage_rate_df

### 2) Работа со столбцом region и housing_type

- **Преобразуем столбец в формат строки**
- **Добавим где нужно данные признаки для корректного объединения в будущем**
- **Убедимся, что все датасеты имеют единый список одних и тех же регионов**
- **Заполним пропущенные значения (NaN)**

In [ ]:
# компоненты, необъодимые для предобработки


In [ ]:
# key_rate
key_rate_df_copy

In [ ]:

# предобработка # Ключевая ставка 
# key_rate_df['date'] = pd.to_datetime(key_rate_df['date'], format='%d.%m.%Y')
# key_rate_df.info()
# min_date_list.append(min(key_rate_df['date']))
# max_date_list.append(max(key_rate_df['date']))

# предобработка # Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)
home_average_mortgage_rate_df = datetime_year_month_df(home_average_mortgage_rate_df)
home_average_mortgage_rate_df.info()
min_date_list.append(min(home_average_mortgage_rate_df['date']))
max_date_list.append(max(home_average_mortgage_rate_df['date']))

# предобработка # Средневзвешенная процентная ставка по ипотечным кредитам, региональный разрез (%)
home_weight_avg_mortgage_rate_df = datetime_year_month_df(home_weight_avg_mortgage_rate_df)
home_weight_avg_mortgage_rate_df.info()
min_date_list.append(min(home_weight_avg_mortgage_rate_df['date']))
max_date_list.append(max(home_weight_avg_mortgage_rate_df['date']))

# предобработка # Средняя цена 1 кв. м общей площади квартир на рынке жилья (рубль) 
home_average_price_df = datetime_year_quarter_df(home_average_price_df)
home_average_price_df = home_average_price_df[home_average_price_df['type_of_housing_market'] != 'Вторичный рынок жилья']
home_average_price_df.drop(columns=['type_of_housing_market'], inplace=True)
home_average_price_df.info()
min_date_list.append(min(home_average_price_df['date']))
max_date_list.append(max(home_average_price_df['date']))

# предобработка # Индексы цен на рынке жилья (%)
home_index_price_df = datetime_year_quarter_df(home_index_price_df)
home_index_price_df = home_index_price_df[home_index_price_df['type_of_housing_market'] != 'Вторичный рынок жилья']
home_index_price_df.drop(columns=['type_of_housing_market'], inplace=True)
home_index_price_df.info()
min_date_list.append(min(home_index_price_df['date']))
max_date_list.append(max(home_index_price_df['date']))

# предобработка # Ключевая ставка + Инфляция (%)
key_rate_inflation_df.drop(columns=['key_rate'], inplace=True)
key_rate_inflation_df['date'] = key_rate_inflation_df['date'].astype(str).str.replace(r'(\d{1,2})\.202\b', r'\1.2020', regex=True)
key_rate_inflation_df['date'] = pd.to_datetime(key_rate_inflation_df['date'], format='%m.%Y')
key_rate_inflation_df.info()
min_date_list.append(min(key_rate_inflation_df['date']))
max_date_list.append(max(key_rate_inflation_df['date']))

# предобработка # Среднедушевые денежные доходы населения (рубль) 
# очистка столбца "year"
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].str[:4]
# очистка столбца 'quarter'
avg_monetary_income_people_df['quarter'] = avg_monetary_income_people_df['quarter'].str.extract(r'(IV|III|II|I)')
avg_monetary_income_people_df = avg_monetary_income_people_df.dropna(subset=['quarter'])
avg_monetary_income_people_df.loc[:, 'quarter']  = avg_monetary_income_people_df['quarter'] + ' квартал'

avg_monetary_income_people_df = datetime_year_quarter_df(avg_monetary_income_people_df)
avg_monetary_income_people_df.info()
min_date_list.append(min(avg_monetary_income_people_df['date']))
max_date_list.append(max(avg_monetary_income_people_df['date']))

# предобработка # Курс доллара
us_rate_df = us_rate_df[['Date', 'Value']].rename(columns={'Date': 'date', 'Value': 'us_rate'})
us_rate_df['date'] = pd.to_datetime(us_rate_df['date'], format='%d.%m.%Y')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2826 entries, 0 to 2825
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      2826 non-null   datetime64[ns]
 1   key_rate  2826 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 44.3 KB
<class 'pandas.core.frame.DataFrame'>
Index: 6912 entries, 1 to 7491
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   region    6912 non-null   object        
 1   avg_rate  6912 non-null   float64       
 2   date      6912 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 216.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 6912 entries, 1 to 7491
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   region           6912 non-null   o

In [13]:
# Выведем списки регионов и посмотрим если повторения
# Унифицируем регионы для каждого датасета и приведем их к единому формату для дальнейшего объединения
all_regions = pd.concat([
    home_average_mortgage_rate_df['region'],
    home_weight_avg_mortgage_rate_df['region'],
    home_average_price_df['region'],
    home_index_price_df['region'],
    avg_monetary_income_people_df['region']
]).unique()

print(all_regions)

['Российская Федерация' '    Центральный федеральный округ'
 '        Белгородская область' '        Брянская область'
 '        Владимирская область' '        Воронежская область'
 '        Ивановская область' '        Калужская область'
 '        Костромская область' '        Курская область'
 '        Липецкая область' '        Московская область'
 '        Орловская область' '        Рязанская область'
 '        Смоленская область' '        Тамбовская область'
 '        Тверская область' '        Тульская область'
 '        Ярославская область' '        г. Москва'
 '    Северо-Западный федеральный округ' '        Республика Карелия'
 '        Республика Коми' '        Архангельская область'
 '            в том числе Ненецкий автономный округ'
 '            Архангельская область без данных по Ненецкому автономному округу'
 '        Вологодская область' '        Калининградская область'
 '        Ленинградская область' '        Мурманская область'
 '        Новгородская область' '   

In [14]:
datasets = [
    home_average_mortgage_rate_df,
    home_weight_avg_mortgage_rate_df,
    home_average_price_df,
    home_index_price_df,
    avg_monetary_income_people_df
]

# убираем лишние пробелы в столбце 'region'
for df in datasets:
    df['region'] = df['region'].str.strip()  # Убираем начальные и конечные пробелы
    df['region'] = df['region'].str.replace(r'\s+', ' ', regex=True)  # Заменяем множественные пробелы на один

all_regions = pd.concat([
    home_average_mortgage_rate_df['region'],
    home_weight_avg_mortgage_rate_df['region'],
    home_average_price_df['region'],
    home_index_price_df['region'],
    avg_monetary_income_people_df['region']
]).unique()

print(all_regions)

['Российская Федерация' 'Центральный федеральный округ'
 'Белгородская область' 'Брянская область' 'Владимирская область'
 'Воронежская область' 'Ивановская область' 'Калужская область'
 'Костромская область' 'Курская область' 'Липецкая область'
 'Московская область' 'Орловская область' 'Рязанская область'
 'Смоленская область' 'Тамбовская область' 'Тверская область'
 'Тульская область' 'Ярославская область' 'г. Москва'
 'Северо-Западный федеральный округ' 'Республика Карелия'
 'Республика Коми' 'Архангельская область'
 'в том числе Ненецкий автономный округ'
 'Архангельская область без данных по Ненецкому автономному округу'
 'Вологодская область' 'Калининградская область' 'Ленинградская область'
 'Мурманская область' 'Новгородская область' 'Псковская область'
 'г. Санкт-Петербург'
 'Южный федеральный округ (до июля 2016г. Республика Крым и г.Севастополь входили в состав Крымского Федерального округа)'
 'Республика Адыгея (Адыгея)' 'Республика Калмыкия' 'Республика Крым'
 'Краснодарск

In [15]:
region_replace_dict = {
    "Российская Федерация": "Россия",
    "Город Москва столица Российской Федерации город федерального значения": "Москва",
    "г. Москва": "Москва",
    "г.Москва": "Москва",
    "Город Санкт-Петербург город федерального значения": "Санкт-Петербург",
    "г. Санкт-Петербург": "Санкт-Петербург",
    "г.Санкт-Петербург": "Санкт-Петербург",
    
    "Центральный федеральный округ": "ЦФО",
    "Северо-Западный федеральный округ": "СЗФО",
    "Южный федеральный округ": "ЮФО",

    "Южный федеральный округ (с 2010 года)": "ЮФО",
    "Южный федеральный округ (с 29.07.2016)": "ЮФО",
    "Южный федеральный округ (до июля 2016г. Республика Крым и г.Севастополь входили в состав Крымского Федерального округа)": "ЮФО",

    "Северо-Кавказский федеральный округ": "СКФО",
    "Приволжский федеральный округ": "ПФО",
    "Уральский федеральный округ": "УФО",
    "Сибирский федеральный округ": "СФО",
    "Сибирский федеральный округ (с 03.11.2018)": "СФО",
    "Сибирский федеральный округ 4)": "СФО",
    "Дальневосточный федеральный округ": "ДФО",
    "Дальневосточный федеральный округ (с 03.11.2018)": "ДФО",
    "Дальневосточный федеральный округ 4)": "ДФО",
    
    "Город федерального значения Севастополь": "Севастополь",
    "г. Севастополь": "Севастополь",
    "г.Севастополь": "Севастополь",
    
    "Крымский федеральный округ": "Республика Крым",
    
    "Ненецкий автономный округ (Архангельская область)": "Ненецкий АО",
    "Ненецкий авт.округ": "Ненецкий АО",
    "в том числе Ненецкий автономный округ": "Ненецкий АО",
    
    "Архангельская область (кроме Ненецкого автономного округа)": "Архангельская область",
    "Архангельская область без данных по Ненецкому автономному округу": "Архангельская область",
    "Архангельская область без авт.округа": "Архангельская область",
    
    "Ханты-Мансийский автономный округ - Югра (Тюменская область)": "ХМАО",
    "Ханты-Мансийский авт.округ": "ХМАО",
    "в том числе Ханты-Мансийский автономный округ - Югра": "ХМАО",
    
    "Ямало-Ненецкий автономный округ (Тюменская область)": "ЯНАО",
    "Ямало-Ненецкий авт.округ": "ЯНАО",
    "в том числе Ямало-Ненецкий автономный округ": "ЯНАО",
    
    "Тюменская область без данных по Ханты-Мансийскому автономному округу - Югре и Ямало-Ненецкому автономному округу": "Тюменская область",
    "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)": "Тюменская область",
    "Тюменская область без авт.округов": "Тюменская область",
    
    "Чувашская Республика - Чувашия": "Чувашская Республика",
    "Чувашская Республика": "Чувашская Республика",
    
    "Республика Татарстан (Татарстан)": "Татарстан",
    "Республика Татарстан": "Татарстан",
    
    "Республика Адыгея (Адыгея)": "Республика Адыгея",
    
    "Республика Бурятия (до конца 2018 года)": "Республика Бурятия",
    "Забайкальский край (до конца 2018 года)": "Забайкальский край",
    
    "Кемеровская область - Кузбасс": "Кемеровская область",
    
    "Еврейская автономная область": "Еврейская АО",
    "Еврейская авт.область": "Еврейская АО",
    
    "Чукотский автономный округ": "Чукотский АО",
    "Чукотский авт.округ": "Чукотский АО",
    "Республика Северная Осетия - Алания": "Республика Северная Осетия",
    "Республика Северная Осетия-Алания": "Республика Северная Осетия",
    "Российская Федерация без учета новых субъектов (с 01.01.2023)":"Россия"
}

# Применение словаря к каждому датасету
for df in datasets:
    df['region'] = df['region'].replace(region_replace_dict)

all_regions = pd.concat([
    home_average_mortgage_rate_df['region'],
    home_weight_avg_mortgage_rate_df['region'],
    home_average_price_df['region'],
    home_index_price_df['region'],
    avg_monetary_income_people_df['region']
]).unique()

print(all_regions)


['Россия' 'ЦФО' 'Белгородская область' 'Брянская область'
 'Владимирская область' 'Воронежская область' 'Ивановская область'
 'Калужская область' 'Костромская область' 'Курская область'
 'Липецкая область' 'Московская область' 'Орловская область'
 'Рязанская область' 'Смоленская область' 'Тамбовская область'
 'Тверская область' 'Тульская область' 'Ярославская область' 'Москва'
 'СЗФО' 'Республика Карелия' 'Республика Коми' 'Архангельская область'
 'Ненецкий АО' 'Вологодская область' 'Калининградская область'
 'Ленинградская область' 'Мурманская область' 'Новгородская область'
 'Псковская область' 'Санкт-Петербург' 'ЮФО' 'Республика Адыгея'
 'Республика Калмыкия' 'Республика Крым' 'Краснодарский край'
 'Астраханская область' 'Волгоградская область' 'Ростовская область'
 'Севастополь' 'СКФО' 'Республика Дагестан' 'Республика Ингушетия'
 'Кабардино-Балкарская Республика' 'Карачаево-Черкесская Республика'
 'Республика Северная Осетия' 'Чеченская Республика' 'Ставропольский край'
 'ПФО' 'Ре

In [16]:
# даты, в диапазоне которых будут данные
start_date = max(min_date_list)
end_date = min(max_date_list)

# объединеняем данные - "Ключевая ставка", "Инфляция", "Курс доллара"
merged_df = key_rate_df.merge(us_rate_df, on="date", how="outer") \
                       .merge(key_rate_inflation_df, on="date", how="outer")

# Заполняем пропуски
merged_df = merged_df.ffill()

# добавляем новый столбец, для дальнейшего объединения
merged_df['region'] = 'Россия'

print(merged_df)

           date  key_rate   us_rate  inflation  region
0    2013-09-17       5.5       NaN        NaN  Россия
1    2013-09-18       5.5       NaN        NaN  Россия
2    2013-09-19       5.5       NaN        NaN  Россия
3    2013-09-20       5.5       NaN        NaN  Россия
4    2013-09-23       5.5       NaN        NaN  Россия
...         ...       ...       ...        ...     ...
3419 2024-12-26      21.0   99,6125       9.52  Россия
3420 2024-12-27      21.0   99,2295       9.52  Россия
3421 2024-12-28      21.0  100,5281       9.52  Россия
3422 2024-12-29      21.0  101,6797       9.52  Россия
3423 2024-12-30      21.0  101,6797       9.52  Россия

[3424 rows x 5 columns]


In [ ]:
# список остальных датасетов
datasets = [
    home_average_mortgage_rate_df,
    home_weight_avg_mortgage_rate_df,
    home_average_price_df,
    home_index_price_df,
    avg_monetary_income_people_df
]

# Последовательно объединяем остальные датасеты по колонкам 'date' и 'region'
for df in datasets[0:]:
    merged_df = pd.merge(merged_df, df, on=['date', 'region'], how='outer')

# Сначала заполнить пропуски и потом срезать по дате
merged_df = merged_df.ffill()
merged_df = merged_df[(merged_df["date"] >= start_date) & (merged_df["date"] <= end_date)]
merged_df = merged_df.sort_values(by='date')

merged_df.to_csv(r'..\data\csv\housing_prices_mortgage_rates_1.csv', index=False, encoding='utf-8')
merged_df



C:\Users\Mi\AppData\Local\Temp\ipykernel_20356\2648690110.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.ffill()


,date,key_rate,us_rate,inflation,region,avg_rate,weight_avg_rate,price,index,income
4493,2019-01-01,NaN,NaN,NaN,Алтайский край,9.87,9.87,44653.65,107.72,20502.222127
4594,2019-01-01,7.75,"69,4706",5.00,Сахалинская область,9.82,9.82,104951.18,129.42,54127.732117
4595,2019-01-01,7.75,"69,4706",5.00,Свердловская область,9.86,9.86,63200.01,105.06,33777.478886
4596,2019-01-01,7.75,"69,4706",5.00,Севастополь,10.49,10.49,65765.90,115.12,21086.930879
4597,2019-01-01,7.75,"69,4706",5.00,Смоленская область,9.90,9.90,40099.94,104.22,24548.672707
...,...,...,...,...,...,...,...,...,...,...
13961,2024-07-01,16.00,"85,7480",8.59,Кировская область,8.51,8.51,112219.50,116.73,43076.000000
13962,2024-07-01,16.00,"85,7480",8.59,Костромская область,8.83,8.83,95204.88,114.60,46395.000000
13963,2024-07-01,16.00,"85,7480",8.59,Краснодарский край,7.80,7.80,150644.14,110.00,62393.000000
13956,2024-07-01,16.00,"85,7480",8.59,Калининградская область,8.16,8.16,121137.26,113.46,47070.000000
